## Installs

In [69]:
# Operational Packages
import numpy as np
import pandas as pd
from pathlib import Path
import sys
import os
import zipfile
import shutil
import geopandas as gpd
from shapely.geometry import Polygon

# Directories
nb_dir = Path.cwd()
REPO_ROOT = nb_dir.parent
data_dir = REPO_ROOT / 'data/'
processed_dir = data_dir / 'processed/'
if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))



In [70]:
# Unzip all Shapefiles
def unzip_to_folder(zip_path, extract_to):
    """
    Unzips a ZIP archive into a specified directory.
    """
    extract_to = Path(extract_to)
    extract_to.mkdir(parents=True, exist_ok=True)

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)                  # Read SHP to destination folder

    macosx_folder = extract_to / '__MACOSX'
    if macosx_folder.exists():
        shutil.rmtree(macosx_folder)

In [71]:
zippaths = Path(data_dir/'raw/zips')                    # ZIPs folders
zipfolders = list(zippaths.glob('*.zip'))           # Select all .ZIP

for folder in zipfolders:
    unzip_to_folder(folder,processed_dir/'shapefiles'/str(folder.name)[:-4])    # Retain name indentity
    print(f'NBAC Wildfires Year: {(str(folder.name)[5:9])} Shapefiles opened')

print(f'Target folder destination: {processed_dir/'shapefiles'}')


NBAC Wildfires Year: 2021 Shapefiles opened
NBAC Wildfires Year: 2020 Shapefiles opened
NBAC Wildfires Year: 2018 Shapefiles opened
NBAC Wildfires Year: 2019 Shapefiles opened
NBAC Wildfires Year: 2022 Shapefiles opened
NBAC Wildfires Year: 2023 Shapefiles opened
NBAC Wildfires Year: 2024 Shapefiles opened
Target folder destination: /Users/mitchellpalmer/Projects/wildfire-risk-analysis/data/processed/shapefiles


In [72]:
print('Open all shapefiles')
# Open all Shapefiles
all_gdfs_dct = {}   # Store in dictionary

for folder in (processed_dir / "shapefiles").iterdir():
    if folder.is_dir():
        shp = next(folder.glob("*.shp"), None)
        if shp:
            name = folder.name
            all_gdfs_dct[name] = gpd.read_file(shp)

# All dictionary keys
print(f'Appended into singular dictionary.')


Open all shapefiles
Appended into singular dictionary.


In [73]:
# Assess each shapefile's (key) column structure

# 1. Get union of all column names
all_cols = sorted(
    {col for gdf in all_gdfs_dct.values() for col in gdf.columns}
)

# 2. Create a DataFrame of False
presence = pd.DataFrame(
    False,
    index=all_gdfs_dct.keys(),
    columns=all_cols,
)

# 3. Mark columns that exist in each GeoDataFrame
for name, gdf in all_gdfs_dct.items():
    presence.loc[name, gdf.columns] = True

presence


,ADJ_FLAG,ADJ_HA,ADMIN_AREA,AG_EDATE,AG_SDATE,BASRC,CAPDATE,FIRECAUS,FIREMAPM,FIREMAPS,...,HS_SDATE,NATPARK,NFIREID,POLY_HA,PRESCRIBED,Shape_Area,Shape_Leng,VERSION,YEAR,geometry
NBAC_2024_20250506,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
NBAC_2023_20250506,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
NBAC_2022_20250506,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
NBAC_2019_20250506,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
NBAC_2018_20250506,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
NBAC_2020_20250506,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
NBAC_2021_20250506,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True


In [74]:
# Assess column structure
missing_per_col = (presence == False).sum(axis=0)      # or (~presence).sum(axis=0)
missing_per_col # Sum False values i.e different structured columns

ADJ_FLAG      0
ADJ_HA        0
ADMIN_AREA    0
AG_EDATE      0
AG_SDATE      0
BASRC         0
CAPDATE       0
FIRECAUS      0
FIREMAPM      0
FIREMAPS      0
GID           0
HS_EDATE      0
HS_SDATE      0
NATPARK       0
NFIREID       0
POLY_HA       0
PRESCRIBED    0
Shape_Area    0
Shape_Leng    0
VERSION       0
YEAR          0
geometry      0
dtype: int64

# Singular GDF Location

In [75]:
# columns that exist in *all* years
common_cols = presence.columns[presence.all(axis=0)]
common_cols


Index(['ADJ_FLAG', 'ADJ_HA', 'ADMIN_AREA', 'AG_EDATE', 'AG_SDATE', 'BASRC',
       'CAPDATE', 'FIRECAUS', 'FIREMAPM', 'FIREMAPS', 'GID', 'HS_EDATE',
       'HS_SDATE', 'NATPARK', 'NFIREID', 'POLY_HA', 'PRESCRIBED', 'Shape_Area',
       'Shape_Leng', 'VERSION', 'YEAR', 'geometry'],
      dtype='object')

In [76]:

# Pick a reference CRS from the first GeoDataFrame
first_gdf = next(iter(all_gdfs_dct.values()))
target_crs = first_gdf.crs

gdfs_to_concat = []

for name, gdf in all_gdfs_dct.items():
    # reproject if needed
    if gdf.crs != target_crs:
        gdf = gdf.to_crs(target_crs)
    
    # either keep only columns shared by all:
    # gdf = gdf[common_cols]

    # or, if you’re okay with missing columns being NaN, skip that line
    gdfs_to_concat.append(gdf)

# Stack them vertically
fires_all_years = gpd.GeoDataFrame(
    pd.concat(gdfs_to_concat, ignore_index=True),
    crs=target_crs
)


In [77]:
combined = {"NBAC_all_years": fires_all_years}
all_gdf_df = combined["NBAC_all_years"]

In [78]:
all_gdf_df

,YEAR,NFIREID,BASRC,FIREMAPS,FIREMAPM,FIRECAUS,HS_SDATE,HS_EDATE,AG_SDATE,AG_EDATE,...,ADJ_HA,ADJ_FLAG,ADMIN_AREA,NATPARK,PRESCRIBED,VERSION,GID,Shape_Leng,Shape_Area,geometry
0,2024.0,1.0,MAFiMS,Landsat,Processed imagery,Natural,2024-08-12,2024-09-02,2024-07-29,2024-10-04,...,625.258308,None,NT,None,None,20250506,2024_1,38513.122912,5.906397e+06,"POLYGON ((-1120513.931 1401415.528, -1120640.6..."
1,2024.0,2.0,MAFiMS,Sentinel-2,Processed imagery,Natural,2024-06-29,2024-08-12,2024-06-28,2024-10-04,...,185.715925,None,NT,None,None,20250506,2024_2,16857.668554,1.754145e+06,"MULTIPOLYGON (((-1245606.340 1450579.000, -124..."
2,2024.0,3.0,MAFiMS,Sentinel-2,Processed imagery,Natural,2024-07-17,2024-07-24,2024-07-17,2024-10-04,...,1188.878170,None,NT,None,None,20250506,2024_3,51547.334192,1.120790e+07,"MULTIPOLYGON (((-846230.854 1422552.091, -8462..."
3,2024.0,4.0,MAFiMS,Landsat,Processed imagery,Natural,2024-07-17,2024-08-13,2024-07-17,2024-10-04,...,3862.626905,None,NT,None,None,20250506,2024_4,132691.511657,3.640874e+07,"MULTIPOLYGON (((-869281.585 1439913.862, -8692..."
4,2024.0,5.0,MAFiMS,Sentinel-2,Processed imagery,Natural,2024-07-28,2024-08-12,2024-07-31,2024-10-04,...,26.552612,None,NT,None,None,20250506,2024_5,2833.473229,2.507250e+05,"POLYGON ((-1229068.442 1459363.850, -1229071.1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10673,2021.0,3012.0,Agency,Undefined,Buffered points,Human,None,None,2021-07-14,None,...,1.370586,true,MB,None,None,20250506,2021_3012,370.034560,1.086572e+04,"POLYGON ((-25063.257 92639.363, -25073.265 926..."
10674,2021.0,3013.0,Agency,Undefined,Undefined,Human,2021-10-03,2021-10-03,2021-05-11,None,...,775.198749,true,MB,None,None,20250506,2021_3013,13953.627354,7.771992e+06,"MULTIPOLYGON (((-379481.043 431792.146, -37948..."
10675,2021.0,3014.0,Agency,Undefined,Undefined,Human,None,None,2021-07-11,None,...,12.005618,true,MB,None,None,20250506,2021_3014,1550.406556,1.002230e+05,"MULTIPOLYGON (((-388861.854 760502.553, -38887..."
10676,2021.0,3015.0,Agency,Undefined,Buffered points,Human,None,None,2021-06-16,None,...,0.001635,true,PC,JA,None,20250506,2021_3015,11.049791,9.691564e+00,"POLYGON ((-1498027.987 706035.643, -1498028.29..."


In [79]:
print(f'All fire geometries dataframe shape: {all_gdf_df.shape} \n')

print(f'NBAC Wildfire value count from years: {all_gdf_df['YEAR'].value_counts().sort_index()}\n')
      


All fire geometries dataframe shape: (10678, 22) 

NBAC Wildfire value count from years: YEAR
2018.0    1698
2019.0     900
2020.0     704
2021.0    1721
2022.0    1479
2023.0    2216
2024.0    1960
Name: count, dtype: int64



In [80]:
print(f'Null Values: \n{all_gdf_df.isnull().sum()}\n')

Null Values: 
YEAR              0
NFIREID           0
BASRC             0
FIREMAPS          0
FIREMAPM          0
FIRECAUS          0
HS_SDATE       6236
HS_EDATE       6236
AG_SDATE        495
AG_EDATE       5195
CAPDATE        3393
POLY_HA           0
ADJ_HA            0
ADJ_FLAG       4559
ADMIN_AREA        0
NATPARK       10228
PRESCRIBED    10618
VERSION           0
GID               0
Shape_Leng        0
Shape_Area        0
geometry          0
dtype: int64



## Geodataframe Cleaning

In [83]:
all_gdf_df.dtypes

YEAR           float64
NFIREID        float64
BASRC           object
FIREMAPS        object
FIREMAPM        object
FIRECAUS        object
HS_SDATE        object
HS_EDATE        object
AG_SDATE        object
AG_EDATE        object
CAPDATE         object
POLY_HA        float64
ADJ_HA         float64
ADJ_FLAG        object
ADMIN_AREA      object
NATPARK         object
PRESCRIBED      object
VERSION         object
GID             object
Shape_Leng     float64
Shape_Area     float64
geometry      geometry
dtype: object

In [84]:
all_gdf_df = all_gdf_df.copy()


col_names = {
    'Shape_Leng' : "shape_length",
    "FIRECAUS": "CAUSE",
    "NFIREID": "FIREID",

}
all_gdf_df = all_gdf_df.rename(columns=col_names)

# lower-case all column names in one line
all_gdf_df.columns = all_gdf_df.columns.str.lower()

all_gdf_df['year'] = all_gdf_df['year'].astype(int)
all_gdf_df['fireid'] = all_gdf_df['fireid'].astype(int)
all_gdf_df['hs_sdate'] = all_gdf_df['hs_sdate'].astype('datetime64[ns]')
all_gdf_df['hs_edate'] = all_gdf_df['hs_edate'].astype('datetime64[ns]')
all_gdf_df['ag_sdate'] = all_gdf_df['ag_sdate'].astype('datetime64[ns]')
all_gdf_df['ag_edate'] = all_gdf_df['ag_edate'].astype('datetime64[ns]')
all_gdf_df['capdate'] = all_gdf_df['capdate'].astype('datetime64[ns]')




## All Fire analysis

In [85]:
all_gdf_df['cause'].value_counts()

cause
Natural         6807
Human           3224
Undetermined     647
Name: count, dtype: int64

In [86]:
all_gdf_df['cause'].value_counts(normalize=True).round(4)*100

cause
Natural         63.75
Human           30.19
Undetermined     6.06
Name: proportion, dtype: float64

In [103]:
all_gdf_df[all_gdf_df['admin_area']=='PC']

,year,fireid,basrc,firemaps,firemapm,cause,hs_sdate,hs_edate,ag_sdate,ag_edate,...,adj_ha,adj_flag,admin_area,natpark,prescribed,version,gid,shape_length,shape_area,geometry
103,2024,101,MAFiMS,Sentinel-2,Processed imagery,Natural,2024-07-07,2024-10-07,2024-07-06,NaT,...,14113.770989,None,PC,NA,None,20250506,2024_101,438358.319478,1.329618e+08,"MULTIPOLYGON (((-1548269.294 1735308.443, -154..."
104,2024,102,MAFiMS,Landsat,Processed imagery,Natural,2024-07-29,2024-09-17,2024-07-29,NaT,...,10058.208435,None,PC,NA,None,20250506,2024_102,218555.636838,9.480882e+07,"MULTIPOLYGON (((-1573165.506 1712493.966, -157..."
288,2024,284,MAFiMS,Sentinel-2,Processed imagery,Natural,2024-06-26,2024-07-05,2024-07-06,2024-08-14,...,186.362618,None,PC,WP,None,20250506,2024_284,17769.537160,1.776284e+06,"POLYGON ((101420.948 902581.485, 101062.144 90..."
332,2024,327,MAFiMS,Sentinel-2,Processed imagery,Natural,2024-08-05,2024-08-13,2024-08-05,NaT,...,827.390283,None,PC,WP,None,20250506,2024_327,49692.705194,7.870744e+06,"MULTIPOLYGON (((122140.332 961087.446, 122099...."
456,2024,451,MAFiMS,Sentinel-2,Processed imagery,Natural,2024-07-23,2024-09-07,2024-07-22,NaT,...,31644.597903,None,PC,JA,None,20250506,2024_451,519511.149662,3.082934e+08,"MULTIPOLYGON (((-1496045.628 673391.102, -1496..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10449,2021,2785,Agency,Undefined,Undefined,Human,NaT,NaT,2021-04-21,NaT,...,63.640685,true,PC,PA,true,20250506,2021_2785,11031.121211,5.738926e+05,"MULTIPOLYGON (((-755266.576 591685.672, -75524..."
10450,2021,2786,Agency,Sentinel-2,Manually delineated,Natural,NaT,NaT,2021-06-04,NaT,...,3.564245,None,PC,WB,None,20250506,2021_2786,1519.748638,3.369948e+04,"MULTIPOLYGON (((-917952.376 1296348.860, -9179..."
10451,2021,2787,Agency,Undefined,Buffered points,Natural,NaT,NaT,2021-07-12,NaT,...,0.393984,true,PC,JA,None,20250506,2021_2787,191.525641,2.913062e+03,"POLYGON ((-1495242.700 689544.710, -1495247.48..."
10676,2021,3015,Agency,Undefined,Buffered points,Human,NaT,NaT,2021-06-16,NaT,...,0.001635,true,PC,JA,None,20250506,2021_3015,11.049791,9.691564e+00,"POLYGON ((-1498027.987 706035.643, -1498028.29..."


In [104]:
# admin_area codes:
# BC, AB, YT, NL, ... and PC for Parks Canada (national parks)

def classify_jurisdiction(row):
    if row["admin_area"] == "PC":
        return "National Park"
    else:
        return row["admin_area"]   # leave provinces/territories as-is

all_gdf_df["jurisdiction"] = all_gdf_df.apply(classify_jurisdiction, axis=1)


In [108]:
all_gdf_df['jurisdiction'].value_counts()

jurisdiction
BC               2753
MB               1936
QC               1545
AB               1214
NT                777
SK                715
YT                494
National Park     450
ON                422
NS                272
NL                 42
NU                 41
NB                  9
PE                  8
Name: count, dtype: int64

#### Canada Fires Export

In [101]:
# Keep only columns you actually need to keep the file small
cols = ['gid', 'fireid', 'year', 'admin_area', 'adj_ha', 'geometry']
Canfires_simple = all_gdf_df[cols].copy()

# 
Canfires_year_min = str(Canfires_simple['year'].min())
Canfires_year_max = str(Canfires_simple['year'].max())

# Reproject to WGS84 for GeoJSON
Canfire_4326 = Canfires_simple.to_crs(epsg=4326)


# Make sure output folder exists
out_dir = processed_dir / "Canada_fires"
out_dir.mkdir(parents=True, exist_ok=True)

# GeoJSON output 
Canfires_path_geojson = processed_dir/ 'Canada_fires'/ f"Canada_fires_{Canfires_year_min}_{Canfires_year_max}.geojson"
Canfire_4326.to_file(Canfires_path_geojson, driver="GeoJSON")

print(f'National Canada Fires extracted between years {Canfires_year_min} - {Canfires_year_max}')
print("Saved Canada_fires GeoJson File", Canfires_path_geojson)

# Shapefile output
Canfires_path_shp = processed_dir/ 'Canada_fires'/f"Canada_fires{Canfires_year_min}_{Canfires_year_max}_shp"
Canfire_4326.to_file(Canfires_path_shp) 

print("Saved Canada_fires Shapefile", Canfires_path_shp)

National Canada Fires extracted between years 2018 - 2024
Saved Canada_fires GeoJson File /Users/mitchellpalmer/Projects/wildfire-risk-analysis/data/processed/Canada_fires/Canada_fires_2018_2024.geojson
Saved Canada_fires Shapefile /Users/mitchellpalmer/Projects/wildfire-risk-analysis/data/processed/Canada_fires/Canada_fires2018_2024_shp


### BC Fires

In [ ]:
BC_gdfs = all_gdf_df[all_gdf_df['admin_area']=='BC']
print(f'BC Wildfires Shape: {BC_gdfs.shape}\n')
BC_gdfs.head(2)

BC Wildfires Shape: (2753, 22)



,year,fireid,basrc,firemaps,firemapm,cause,hs_sdate,hs_edate,ag_sdate,ag_edate,...,adj_ha,adj_flag,admin_area,natpark,prescribed,version,gid,shape_length,shape_area,geometry
162,2024,159,MAFiMS,Landsat,Processed imagery,Natural,2024-05-09,2024-07-19,2024-05-10,None,...,12260.052718,None,BC,None,None,20250506,2024_159,8.001664e+05,1.164338e+08,"MULTIPOLYGON (((-1488758.227 1319811.062, -148..."
163,2024,160,MAFiMS,Sentinel-2,Processed imagery,Natural,2024-04-11,2024-10-16,2023-06-28,None,...,450940.307996,None,BC,None,None,20250506,2024_160,1.599471e+07,4.271401e+09,"MULTIPOLYGON (((-1421109.650 1317743.467, -142..."


#### BC Fires Analysis

In [ ]:
BC_gdfs['cause'].value_counts()

cause
Natural         1916
Human            795
Undetermined      42
Name: count, dtype: int64

In [ ]:
BC_gdfs['cause'].value_counts(normalize=True)*100

cause
Natural         69.596803
Human           28.877588
Undetermined     1.525608
Name: proportion, dtype: float64

#### BC Fires Export

In [ ]:
# Keep only columns you actually need to keep the file small
cols = ['gid', 'fireid', 'year', 'admin_area', 'adj_ha', 'geometry']
BC_simple = BC_gdfs[cols].copy()

# 
BC_year_min = str(BC_simple['year'].min())
BC_year_max = str(BC_simple['year'].max())

# Reproject to WGS84 for GeoJSON
BC_4326 = BC_simple.to_crs(epsg=4326)


# Make sure output folder exists
out_dir = processed_dir / "BC_fires"
out_dir.mkdir(parents=True, exist_ok=True)

# GeoJSON output 
bc_path_geojson = processed_dir/ 'BC_fires'/ f"BC_fires_{BC_year_min}_{BC_year_max}.geojson"
BC_4326.to_file(bc_path_geojson, driver="GeoJSON")

print(f'BC Fires extracted between years {BC_year_min} - {BC_year_max}')
print("Saved BC_Fires GeoJson File", bc_path_geojson)

# Shapefile output
bc_path_shp = processed_dir/ 'BC_fires'/f"BC_fires_{BC_year_min}_{BC_year_max}_shp"
BC_4326.to_file(bc_path_shp) 

print("Saved BC_Fires Shapefile", bc_path_geojson)

BC Fires extracted between years 2018 - 2024
Saved BC_Fires GeoJson File /Users/mitchellpalmer/Projects/wildfire-risk-analysis/data/processed/BC_fires/BC_fires_2018_2024.geojson
Saved BC_Fires Shapefile /Users/mitchellpalmer/Projects/wildfire-risk-analysis/data/processed/BC_fires/BC_fires_2018_2024.geojson


In [95]:
all_gdf_df[all_gdf_df['year']==2024]

,year,fireid,basrc,firemaps,firemapm,cause,hs_sdate,hs_edate,ag_sdate,ag_edate,...,adj_ha,adj_flag,admin_area,natpark,prescribed,version,gid,shape_length,shape_area,geometry
0,2024,1,MAFiMS,Landsat,Processed imagery,Natural,2024-08-12,2024-09-02,2024-07-29,2024-10-04,...,625.258308,None,NT,None,None,20250506,2024_1,38513.122912,5.906397e+06,"POLYGON ((-1120513.931 1401415.528, -1120640.6..."
1,2024,2,MAFiMS,Sentinel-2,Processed imagery,Natural,2024-06-29,2024-08-12,2024-06-28,2024-10-04,...,185.715925,None,NT,None,None,20250506,2024_2,16857.668554,1.754145e+06,"MULTIPOLYGON (((-1245606.340 1450579.000, -124..."
2,2024,3,MAFiMS,Sentinel-2,Processed imagery,Natural,2024-07-17,2024-07-24,2024-07-17,2024-10-04,...,1188.878170,None,NT,None,None,20250506,2024_3,51547.334192,1.120790e+07,"MULTIPOLYGON (((-846230.854 1422552.091, -8462..."
3,2024,4,MAFiMS,Landsat,Processed imagery,Natural,2024-07-17,2024-08-13,2024-07-17,2024-10-04,...,3862.626905,None,NT,None,None,20250506,2024_4,132691.511657,3.640874e+07,"MULTIPOLYGON (((-869281.585 1439913.862, -8692..."
4,2024,5,MAFiMS,Sentinel-2,Processed imagery,Natural,2024-07-28,2024-08-12,2024-07-31,2024-10-04,...,26.552612,None,NT,None,None,20250506,2024_5,2833.473229,2.507250e+05,"POLYGON ((-1229068.442 1459363.850, -1229071.1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1955,2024,1950,Agency,Undefined,Buffered points,Human,NaT,NaT,2024-11-16,2024-11-16,...,0.045019,true,QC,None,None,20250506,2024_1950,63.684514,3.225389e+02,"POLYGON ((1621590.722 -120278.608, 1621589.838..."
1956,2024,1951,Agency,Undefined,Buffered points,Undetermined,NaT,NaT,2024-12-31,2025-01-02,...,0.533702,true,MB,None,None,20250506,2024_1951,225.205593,4.025725e+03,"POLYGON ((-443478.206 303507.381, -443484.462 ..."
1957,2024,1952,MAFiMS,Sentinel-2,Processed imagery,Undetermined,NaT,NaT,NaT,NaT,...,32.829829,None,NU,None,None,20250506,2024_1952,2977.691737,3.086130e+05,"POLYGON ((-267478.974 1768331.248, -267479.779..."
1958,2024,1953,MAFiMS,Sentinel-2,Processed imagery,Undetermined,NaT,NaT,NaT,NaT,...,2.571199,None,NU,None,None,20250506,2024_1953,748.345289,2.416507e+04,"POLYGON ((174126.403 1675674.317, 174116.983 1..."


In [ ]:
polygon = [-118.49342925291688,52.9012441034348]
[-118.13637358885438,52.49846239890529]
[-117.50465972166688,52.32422286911985]
[-117.22450835447938,52.421479808374734]
[-117.27943999510438,52.871413013056674]
[-117.94960601072938,53.19184541828504]
[-118.49342925291688,52.9012441034348]

In [96]:
# 2024 fires in Alberta
fires_2024_ab = all_gdf_df.query("year == 2024 and admin_area == 'AB'").copy()

print(f"2024 fires in AB: {len(fires_2024_ab)}")

# See the big ones first
fires_2024_ab.sort_values("adj_ha", ascending=False).head(20)[
    ["gid", "fireid", "year", "adj_ha", "admin_area", "natpark",
     "hs_sdate", "hs_edate", "ag_sdate", "ag_edate"]
]


2024 fires in AB: 231


,gid,fireid,year,adj_ha,admin_area,natpark,hs_sdate,hs_edate,ag_sdate,ag_edate
159,2024_156,156,2024,177114.294714,AB,None,2024-06-29,2024-09-24,2024-06-29,NaT
135,2024_133,133,2024,84938.906193,AB,None,2024-07-03,2024-08-17,2024-07-03,NaT
23,2024_24,24,2024,81133.988043,AB,None,2024-05-06,2024-08-13,2024-06-01,2024-10-04
149,2024_147,147,2024,58137.180823,AB,None,2024-07-02,2024-09-19,2024-07-02,NaT
144,2024_142,142,2024,29427.983590,AB,None,2024-07-02,2024-09-17,2024-07-02,NaT
151,2024_149,149,2024,28754.247436,AB,None,2024-04-21,2024-08-16,2024-04-21,NaT
157,2024_155,155,2024,25016.467177,AB,None,2024-07-08,2024-08-17,2024-07-08,2024-09-06
57,2024_56,56,2024,24362.005793,AB,None,2024-04-20,2024-09-17,2024-05-24,2024-11-14
124,2024_122,122,2024,21483.701861,AB,None,2024-07-06,2024-08-16,2024-07-05,NaT
153,2024_151,151,2024,15818.767670,AB,None,2024-07-11,2024-08-16,2024-07-11,NaT


In [97]:
import geopandas as gpd
from shapely.geometry import Polygon

# Your polygon (lon, lat) pairs
jasper_coords = [
    (-118.49342925291688, 52.9012441034348),
    (-118.13637358885438, 52.49846239890529),
    (-117.50465972166688, 52.32422286911985),
    (-117.22450835447938, 52.421479808374734),
    (-117.27943999510438, 52.871413013056674),
    (-117.94960601072938, 53.19184541828504),
    (-118.49342925291688, 52.9012441034348),
]

jasper_poly = Polygon(jasper_coords)

jasper_gdf = gpd.GeoDataFrame(
    {"aoi": ["Jasper_AOI"]},
    geometry=[jasper_poly],
    crs="EPSG:4326"        # coords you pasted are lon/lat
)


In [98]:
# Filter to 2024 fires first
fires_2024 = all_gdf_df[all_gdf_df["year"] == 2024].copy()

# Ensure they are in the same CRS as the Jasper polygon
if fires_2024.crs is None:
    print("WARNING: all_gdf_df has no CRS set – set it before continuing.")
elif fires_2024.crs.to_epsg() != 4326:
    fires_2024 = fires_2024.to_crs(jasper_gdf.crs)

# Spatial join: which fires intersect your Jasper AOI?
fires_2024_jasper = gpd.sjoin(
    fires_2024,
    jasper_gdf,
    how="inner",
    predicate="intersects"
)

print(f"Fires intersecting Jasper AOI in 2024: {len(fires_2024_jasper)}")

fires_2024_jasper[
    ["gid", "fireid", "year", "adj_ha", "admin_area",
     "natpark", "hs_sdate", "hs_edate", "ag_sdate", "ag_edate"]
]


Fires intersecting Jasper AOI in 2024: 2


,gid,fireid,year,adj_ha,admin_area,natpark,hs_sdate,hs_edate,ag_sdate,ag_edate
456,2024_451,451,2024,31644.597903,PC,JA,2024-07-23,2024-09-07,2024-07-22,NaT
726,2024_719,719,2024,0.328154,PC,JA,NaT,NaT,2024-07-18,NaT
